In [ ]:
from matplotlib.pyplot import figure

pd.set_option('display.max_rows', 500)

dfLinks = pd.read_csv('EVA_Links.csv', header=None, encoding='utf8')
dfLinks.columns = ['i','j']
dfNodes = pd.read_csv('EVA_nodename.csv', header=None, encoding='utf8', skip_blank_lines=False)  

#create an adjacencyList for owners
adjacencyList = [[] for n in range(len(dfNodes) + 1)]

for _, row in dfLinks.iterrows():
    i = row["i"]
    j = row['j']
    adjacencyList[j].append(i)

def stochasticMatrix(adjacency_list):
    n = len(adjacency_list)
    matrix = np.zeros((n, n))
    for col, line in enumerate(adjacency_list):
        if len(line) > 0:
              #page has existing links
            for index in line:
                matrix[index][col] = 1/(len(line))
        else:
            #page not found
            for index in range(n):
                matrix[index][col] = 1/n
    return matrix

#transitionMatrix = (1-beta)/n * m + beta*stochasitcMatrix
def transitionMatrix(stochastic_matrix):
    n = len(stochastic_matrix)
    beta = 0.85 #teleportation factor
    m = np.ones((n, n)) #matrix of 1's
    part1 = np.multiply(((1-beta)/n), m)
    part2 = np.multiply(beta, stochastic_matrix)
    transition_matrix = np.add(part1, part2)
    return transition_matrix

def pageRank(transition_matrix):
    n = len(transition_matrix)
    err = 0.0005 
    v1 = np.ones(n)
    v1 = np.multiply((1/n), v1)
    v2 = np.matmul(transition_matrix, v1)
    
    while not within_err(v1, v2, err): 
        #ensures differences between v1 and v2 is under the err bound
        v1 = v2
        v2 = np.matmul(transition_matrix, v1)
    return (v2.tolist())

def within_err(v1, v2, err):
    diff_vector = np.subtract(v2, v1)
    for diff in diff_vector:
        if abs(diff) > err:
            return False
    return True


m = stochasticMatrix(adjacencyList)
#print(m)

t = transitionMatrix(m)
#print(t)

p = pageRank(t)
#print(p)
#print(len(p))

N = 10
#Return index values(company ids) for the 10 highest page ranks
topCompanyID = sorted(range(len(p)), key = lambda sub: p[sub])[-N:]
compList = dfNodes.to_numpy() #convert df to array

#Top Company Name Lookups
print("Top 10 Companies")
#print(compList[0])
for i in topCompanyID:
    print(compList[i - 1])
#print(compList[topCompanyID])

